In [1]:
import numpy as np
import pandas as pd

c:\Users\89556\.conda\envs\pydata\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# 数据转换

## 删除重复数据

In [2]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [6]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [7]:
data.drop_duplicates(subset=['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [8]:
data.drop_duplicates(['k1','k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 利用函数或映射进行数据转换

In [9]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [10]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

In [11]:
data['animal'] = data['food'].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [12]:
def get_animal(x):
    return meat_to_animal[x]

In [13]:
data['food'].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## 替换值

In [14]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [15]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [16]:
data.replace([-999,1000], np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [17]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [18]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 重命名轴索引

In [19]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

In [20]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [21]:
def transform(X):
    return X[:4].upper()

In [22]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [24]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [25]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [26]:
data.rename(index={'OHIO':'INDIANA'},
            columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 离散化和分箱

In [27]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [28]:
bins = [18,25,35,60,100]

In [30]:
age_categories = pd.cut(ages, bins)

age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [31]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [32]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [33]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [34]:
pd.value_counts(age_categories)

C:\Users\89556\AppData\Local\Temp\ipykernel_36340\3010498523.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

In [35]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [36]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [37]:
data = np.random.uniform(size=20)
data

array([0.06984776, 0.19930254, 0.04029134, 0.92017023, 0.22955513,
       0.45687985, 0.54269469, 0.22015523, 0.12312696, 0.97943417,
       0.73958803, 0.72698504, 0.32767531, 0.53539354, 0.77775106,
       0.07668929, 0.46668323, 0.47494245, 0.85299136, 0.88791625])

In [38]:
pd.cut(data,4,precision=2)

[(0.039, 0.28], (0.039, 0.28], (0.039, 0.28], (0.74, 0.98], (0.039, 0.28], ..., (0.039, 0.28], (0.28, 0.51], (0.28, 0.51], (0.74, 0.98], (0.74, 0.98]]
Length: 20
Categories (4, interval[float64, right]): [(0.039, 0.28] < (0.28, 0.51] < (0.51, 0.74] < (0.74, 0.98]]

In [39]:
data = np.random.standard_normal(1000)
data

array([ 1.06327107e+00,  3.85458816e-01,  1.98442765e-01, -1.82847176e+00,
       -1.14500818e+00,  4.02181439e-01,  2.21207985e-02, -1.01865451e+00,
        9.42599921e-02,  6.14232384e-01,  1.00831338e-01,  1.09987238e+00,
       -1.40382430e-01,  1.76104979e-01, -1.91995590e+00, -3.52749897e-01,
        2.78794984e-01, -7.48629092e-01,  3.83955145e-01,  6.82804434e-01,
       -1.05660662e+00,  7.95095307e-01, -7.44524605e-01, -1.56317007e+00,
        9.85924505e-03,  1.29256494e+00, -9.03630358e-01, -8.04594798e-01,
        4.23335466e-01, -1.73378133e+00, -9.63633560e-01, -6.13579136e-01,
       -9.22416088e-01,  9.27675239e-01,  5.16191536e-01, -9.66332990e-01,
       -2.20983053e+00, -2.75872515e-01,  2.25523564e-01, -1.98778392e-01,
        8.95675347e-01, -1.06902528e+00, -1.23286470e-01,  7.22449253e-01,
        8.46224230e-01,  6.04252868e-02, -4.75109435e-01, -4.03397559e-02,
        1.23859714e+00,  1.00174577e+00, -1.32365638e+00,  3.96568014e-01,
       -1.86309056e-01, -

In [40]:
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.67, 3.12], (0.022, 0.67], (0.022, 0.67], (-2.9, -0.72], (-2.9, -0.72], ..., (0.67, 3.12], (-2.9, -0.72], (0.022, 0.67], (0.022, 0.67], (-0.72, 0.022]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9, -0.72] < (-0.72, 0.022] < (0.022, 0.67] < (0.67, 3.12]]

In [41]:
pd.value_counts(quartiles)

C:\Users\89556\AppData\Local\Temp\ipykernel_36340\3472704981.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


(-2.9, -0.72]     250
(-0.72, 0.022]    250
(0.022, 0.67]     250
(0.67, 3.12]      250
Name: count, dtype: int64

In [42]:
pd.qcut(data,q=[0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.895, -1.291]    100
(-1.291, 0.0219]    400
(0.0219, 1.307]     400
(1.307, 3.119]      100
Name: count, dtype: int64

## 检测和过滤异常值

In [48]:
data = pd.DataFrame(np.random.standard_normal((1000,4)))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.039454,-0.036982,0.044259,0.038391
std,0.968075,0.983761,1.023245,0.996786
min,-3.106627,-3.295916,-3.433710,-3.073357
25%,-0.699981,-0.690549,-0.637777,-0.666484
50%,-0.045239,-0.024084,0.048570,0.036582
75%,0.609515,0.605793,0.735484,0.678169
max,3.163747,2.917209,2.755586,3.234486


In [49]:
col = data[2]

In [50]:
col[col.abs()>3]

148   -3.272335
225   -3.110891
947   -3.433710
Name: 2, dtype: float64

In [51]:
data[(data.abs()>3).any(axis='columns')]

,0,1,2,3
13,-0.648557,-0.832875,-0.679235,-3.073357
33,3.163747,1.138627,-0.557788,0.165835
73,-1.788994,-3.295916,-0.208570,1.638487
148,0.076846,0.194928,-3.272335,-1.040419
225,-0.322562,-1.800585,-3.110891,-1.896953
332,-3.106627,0.555091,0.671089,-1.571477
356,-0.087782,0.818327,-0.201055,3.064923
622,1.373286,-2.667502,0.229121,3.234486
659,3.002807,-0.527970,-0.424943,1.008211
947,-0.031744,-0.426388,-3.433710,-0.190504


In [52]:
data[data.abs()>3] = np.sign(data)*3

In [53]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.039514,-0.036686,0.045076,0.038164
std,0.967205,0.982824,1.020674,0.995639
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.699981,-0.690549,-0.637777,-0.666484
50%,-0.045239,-0.024084,0.048570,0.036582
75%,0.609515,0.605793,0.735484,0.678169
max,3.000000,2.917209,2.755586,3.000000


In [54]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0
3,-1.0,1.0,-1.0,-1.0
4,1.0,1.0,1.0,-1.0


## 置换和随机采样

In [56]:
df = pd.DataFrame(np.arange(5*7).reshape((5,7)))

df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [89]:
sampler = np.random.permutation(5)

sampler

array([2, 4, 1, 3, 0])

In [90]:
df.take(sampler)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [91]:
df.iloc[sampler]

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


In [96]:
column_sample = np.random.permutation(7)
column_sample

array([2, 1, 4, 5, 6, 0, 3])

In [97]:
df.take(column_sample,axis='columns')

,2,1,4,5,6,0,3
0,2,1,4,5,6,0,3
1,9,8,11,12,13,7,10
2,16,15,18,19,20,14,17
3,23,22,25,26,27,21,24
4,30,29,32,33,34,28,31


In [114]:
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


In [118]:
choices = pd.Series([5,7,-1,6,4])

choices.sample(n=10,replace=True)

3    6
0    5
1    7
1    7
3    6
1    7
2   -1
1    7
2   -1
3    6
dtype: int64

## 计算指标/虚拟变量

In [120]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [121]:
pd.get_dummies(df['key'])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [122]:
dummies = pd.get_dummies(df['key'],prefix='key')

In [123]:
df_with_dummy = df[['data1']].join(dummies)

In [124]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


In [126]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [127]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("../datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [128]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


In [129]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       